In [4]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='6d41f7e9c2a24e9395e7da1f9a7b8bc0' name='default'>

In [11]:
from aiida_kkr.calculations import KkrimpCalculation
from masci_tools.io.parsers.kkrimp_parser_functions import KkrimpParserFunctions

SyntaxError: invalid syntax (kkrimp_parser_functions.py, line 152)

In [7]:
critical_files = [
            ('outfile', KkrimpCalculation._DEFAULT_OUTPUT_FILE),
            ('out_log', KkrimpCalculation._OUTPUT_000),
            ('out_pot', KkrimpCalculation._OUT_POTENTIAL),
            ('out_timing', KkrimpCalculation._OUT_TIMING_000),
            ('out_enersp_at', KkrimpCalculation._OUT_ENERGYSP_PER_ATOM),
            ('out_enertot_at', KkrimpCalculation._OUT_ENERGYTOT_PER_ATOM),
        ]

In [8]:
ret = orm.load_node(43648).outputs.retrieved
for name in critical_files:
    print(name)
    with ret.open(name[1], 'rb') as _f:
        with open('test_files/'+name[1], 'wb') as _f2:
            _f2.writelines(_f.readlines())

NotExistent: no Node found with ID<43648>: No result was found

In [9]:
named_file_handles = {key: 'test_files/'+val for key, val in critical_files}
# named_file_handles['out_log'] = 'test_files/out_log.000.txt_small'
named_file_handles

{'outfile': 'test_files/out_kkrimp',
 'out_log': 'test_files/out_log.000.txt',
 'out_pot': 'test_files/out_potential',
 'out_timing': 'test_files/out_timing.000.txt',
 'out_enersp_at': 'test_files/out_energysp_per_atom_eV',
 'out_enertot_at': 'test_files/out_energytotal_per_atom_eV'}

In [10]:
# now we can parse the output files
success, msg_list, out_dict = KkrimpParserFunctions().parse_kkrimp_outputfile(
    {}, named_file_handles, debug=False, timings=True
)

NameError: name 'KkrimpParserFunctions' is not defined

In [4]:
from masci_tools import __file__

In [5]:
print(__file__)

/opt/masci-tools/masci_tools/__init__.py


In [12]:
from masci_tools.io.parsers.kkrparser_functions import get_rms, parse_array_float

def test():
    rms_charge, rms_spin, result_atoms_last_charge, result_atoms_last_spin = get_rms(named_file_handles['outfile'],
                                                                                     named_file_handles['out_log'],
                                                                                     debug=False)
    
%timeit test()

1.39 s ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
debug = False
def test():
    rms_charge = parse_array_float(named_file_handles['outfile'], 'average rms-error', [2, '=', 1, 0], ['D', 'E'], debug=debug)

%timeit test()

297 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:

def test():
    rms_charge_atoms = parse_array_float(named_file_handles['out_log'], 'rms-error for atom', [2, '=', 1, 0], ['D', 'E'])

%timeit test()

408 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
from masci_tools.io.parsers.kkrparser_functions import get_outfile_txt, search_string
from masci_tools.io.common_functions import search_string_generator

In [13]:
outfile = named_file_handles['outfile']

%timeit tmptxt = get_outfile_txt(outfile)

1.64 ms ± 7.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
tmptxt = get_outfile_txt(outfile)

In [16]:
searchstring = 'average rms-error'
replacepair = ['D', 'E']
splitinfo = [2, '=', 1, 0]

debug = False
import numpy as np

def test():
    itmp = 0
    res = []
    # print(len(tmptxt))
    while itmp >= 0:
        itmp = search_string(searchstring, tmptxt)
        if debug:
            print(('in parse_array_float (itmp, searchstring, outfile):', itmp, searchstring, outfile))
        if itmp >= 0:
            tmpval = tmptxt.pop(itmp)
            if replacepair is not None:
                tmpval = tmpval.replace(replacepair[0], replacepair[1])
            if splitinfo[0] == 1:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]])
            elif splitinfo[0] == 2:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
            else:
                raise ValueError('splitinfo[0] has to be either 1 or 2')
            res.append(tmpval)
    res = np.array(res)
    return res
    
%timeit res = test()

2 ms ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
import numpy as np

# def search_new(tmptxt, searchstring, splitinfo, replacepair=None, debug=True):
#     itmp = 1
#     res = []
#     while itmp in search_string_generator(searchstring, tmptxt):
#         if debug:
#             print(('in parse_array_float (itmp, searchstring, outfile):', itmp[0], searchstring, outfile))
#         if itmp[0] >= 0:
#             if replacepair is not None:
#                 itmp[1] = itmp[1].replace(replacepair[0], replacepair[1])
#             if splitinfo[0] == 1:
#                 itmp[1] = float(itmp[1].split(splitinfo[1])[splitinfo[2]])
#             elif splitinfo[0] == 2:
#                 itmp[1] = float(itmp[1].split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
#             else:
#                 raise ValueError('splitinfo[0] has to be either 1 or 2')
#             res.append(itmp[1])
#     res = np.array(res)
#     return res

def search_new(tmptxt, searchstring, replacepair, splitinfo, debug=False):
    res = []
    for (itmp, tmpval) in search_string_generator(searchstring, tmptxt):
        if debug:
            print(('in parse_array_float (itmp, searchstring, outfile):', itmp, tmpval, searchstring, outfile))
        if itmp >= 0:
            if replacepair is not None:
                if debug:
                    print('replace', tmpval, type(tmpval), replacepair[0], replacepair[1])
                tmpval = tmpval.replace(replacepair[0], replacepair[1])
            if splitinfo[0] == 1:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]])
            elif splitinfo[0] == 2:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
            else:
                raise ValueError('splitinfo[0] has to be either 1 or 2')
            res.append(tmpval)
    res = np.array(res)
    return res


def test_new():
    tmptxt = get_outfile_txt(outfile)
    res = search_new(tmptxt, searchstring, replacepair, splitinfo)
    return res
    
%timeit res = test_new()
test_new()

4.61 ms ± 5.62 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


array([1.8268 , 1.7993 , 1.7741 , 1.7494 , 1.7252 , 1.4396 , 1.6901 ,
       1.66   , 1.6316 , 1.6098 , 1.5893 , 1.5692 , 1.5495 , 1.5301 ,
       1.5113 , 1.4928 , 1.4749 , 1.4573 , 1.4402 , 1.4235 , 1.4072 ,
       1.3913 , 1.3757 , 1.3605 , 1.3457 , 1.3312 , 1.317  , 1.3031 ,
       1.2896 , 1.2763 , 1.2634 , 1.2507 , 1.2383 , 1.2261 , 1.2142 ,
       1.2025 , 1.1911 , 1.1799 , 1.169  , 1.1583 , 1.1478 , 1.1375 ,
       1.1274 , 1.1175 , 1.1078 , 1.0983 , 1.089  , 1.0798 , 1.0708 ,
       1.0619 , 1.0532 , 1.0446 , 1.0362 , 1.0279 , 1.0198 , 1.0118 ,
       1.0039 , 0.99619, 0.98858, 0.98111, 0.97375, 0.96652, 0.95939,
       0.95237, 0.94544, 0.93861, 0.93185, 0.92516, 0.91855, 0.912  ,
       0.90547, 0.89901, 0.89274, 0.8866 , 0.88039, 0.87338, 0.86353,
       0.84973, 0.87079, 1.0943 , 1.5694 , 1.8131 , 1.7771 , 1.7545 ,
       1.734  , 1.713  , 1.6913 , 1.6689 , 1.6455 , 1.6207 , 1.5939 ,
       1.5642 , 1.53   , 1.4885 , 1.4346 , 1.3579 , 1.2437 , 1.0821 ,
       0.89237, 0.72

In [41]:
for i in search_string_generator('DO NOT FIND ME', tmptxt):
    print('in loop')

In [40]:
def search(tmptxt, searchstring, replacepair, splitinfo):
    itmp = 0
    res = []
    while itmp >= 0:
        itmp = search_string(searchstring, tmptxt)
        if debug:
            print(('in parse_array_float (itmp, searchstring, outfile):', itmp, searchstring, outfile))
        if itmp >= 0:
            tmpval = tmptxt.pop(itmp)
            if replacepair is not None:
                tmpval = tmpval.replace(replacepair[0], replacepair[1])
            if splitinfo[0] == 1:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]])
            elif splitinfo[0] == 2:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
            else:
                raise ValueError('splitinfo[0] has to be either 1 or 2')
            res.append(tmpval)
    res = np.array(res)
    return res

def test_old():
    tmptxt = get_outfile_txt(outfile)
    res = search(tmptxt, searchstring, replacepair, splitinfo)
    return res
    
%timeit res = test_old()
# test_old()

289 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
test_old() == test_new()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [41]:

tmptxt = get_outfile_txt(outfile)
def test():
    res = search(tmptxt, searchstring, replacepair, splitinfo)
    return res
    
%timeit res = test()

1.92 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
